In [1]:
import amostra
import pandas
import amostra.client.commands as acc
import collections

In [2]:
# Definitions
owner = "hhslepicka"
project = "12345"
beamline_id = "LIX"

In [3]:
container_ref = acc.ContainerReference()
sample_ref = acc.SampleReference()

In [4]:
def insert_plate(**plate_info):
    plate = container_ref.create(**plate_info)
    return plate

def insert_sample_list(samples):
    result = collections.deque()
    for s in samples:
        result.append(sample_ref.create(**s))
    
    return result

In [5]:
def generate_plate(owner, project, beamline_id, kind, name, barcode, content):
    plate_info = {    
            "owner": owner,
            "project": project,
            "beamline_id": beamline_id,
            "kind": kind,
            "name": name,
            "barcode": barcode,
            "content": list(content)
    }

    return insert_plate(**plate_info)

def generate_samples_96wp(project, beamline_id, owner, name):
    samples = collections.deque()
    for i in range(96):
        sample_info = {
            "project": project,
            "beamline_id": beamline_id,
            "owner": owner,
            "name": name+" {}".format(i+1),
            "position": {"x": int(i % 8), "y": int(i/8)},
            "concentration": 0.001,
            "volume": 10,
            "temperature": 21.4    
        }
        s = sample_ref.create(**sample_info)
        samples.append(s)
    return samples

def find_plate_by_barcode(owner, project, beamline_id, barcode):
    samples = collections.deque()
    plate_info = list(container_ref.find(owner=owner, project=project, beamline_id=beamline_id, barcode=barcode))[0]
    for s_uid in plate_info['content']:
        samples.append(next(sample_ref.find(uid=s_uid)))
    
    plate_info['content'] = list(samples)
    return plate_info

In [6]:
number_of_plates = 10

In [7]:
for i in range(number_of_plates):
    samples = generate_samples_96wp(project, beamline_id, owner, "Test Sample")
    plate1 = generate_plate(owner, project, beamline_id, "96wp", "Plate {}".format(i+1), "{}".format(i+1).zfill(13), samples)

In [ ]:
-

In [ ]:
excel_file = "Sample_Import.xlsx"
plate, samples = import_plate_from_excel(excel_file, owner, project, beamline_id, "96wp")

In [ ]:
content = [s['uid'] for s in insert_sample_list(samples)]
plate['content'] = content
inserted_plate = insert_plate(plate)

In [ ]:
inserted_plate

In [ ]:
r = find_plate_by_barcode(owner, project, beamline_id, "1000000000001".zfill(13))

In [ ]:
r["name"]